# Entity Resolution Companies

In [1]:
!pip install recordlinkage --quiet
!pip install sklearn --quiet

In [2]:
import numpy as np 
import pandas as pd
import sklearn
import recordlinkage
import warnings
from recordlinkage.index import Full
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

In [1]:
companies = pd.read_csv("alignedSchemas/companiesAligned.csv")
companies.head(10)

NameError: name 'pd' is not defined

In [4]:
######

from recordlinkage.datasets import load_febrl4

dfA, dfB, true_links = load_febrl4(return_links=True)

print(true_links)

MultiIndex([(   'rec-0-org',    'rec-0-dup-0'),
            (   'rec-1-org',    'rec-1-dup-0'),
            (   'rec-2-org',    'rec-2-dup-0'),
            (   'rec-3-org',    'rec-3-dup-0'),
            (   'rec-4-org',    'rec-4-dup-0'),
            (   'rec-5-org',    'rec-5-dup-0'),
            (   'rec-6-org',    'rec-6-dup-0'),
            (   'rec-7-org',    'rec-7-dup-0'),
            (   'rec-8-org',    'rec-8-dup-0'),
            (   'rec-9-org',    'rec-9-dup-0'),
            ...
            ('rec-4990-org', 'rec-4990-dup-0'),
            ('rec-4991-org', 'rec-4991-dup-0'),
            ('rec-4992-org', 'rec-4992-dup-0'),
            ('rec-4993-org', 'rec-4993-dup-0'),
            ('rec-4994-org', 'rec-4994-dup-0'),
            ('rec-4995-org', 'rec-4995-dup-0'),
            ('rec-4996-org', 'rec-4996-dup-0'),
            ('rec-4997-org', 'rec-4997-dup-0'),
            ('rec-4998-org', 'rec-4998-dup-0'),
            ('rec-4999-org', 'rec-4999-dup-0')],
           length=5000)

In [5]:
#train, test = train_test_split(companies, test_size=0.2, random_state=1)

In [6]:
indexer = recordlinkage.Index()
indexer.block('name')
pairs = indexer.index(companies, companies)
print(pairs)

indexer2 = recordlinkage.Index()
indexer2.add(Full())
allpairs = indexer2.index(companies, companies)
print(allpairs)

MultiIndex([(    0,     0),
            (    0, 20812),
            (    0, 44061),
            (    0, 62588),
            (20812,     0),
            (20812, 20812),
            (20812, 44061),
            (20812, 62588),
            (44061,     0),
            (44061, 20812),
            ...
            (64593, 64593),
            (64594, 64594),
            (64595, 64595),
            (64596, 64596),
            (64598, 64598),
            (64601, 64601),
            (64602, 64602),
            (64603, 64603),
            (64606, 64606),
            (64607, 64607)],
           length=178227)
MultiIndex([(    0,     0),
            (    0,     1),
            (    0,     2),
            (    0,     3),
            (    0,     4),
            (    0,     5),
            (    0,     6),
            (    0,     7),
            (    0,     8),
            (    0,     9),
            ...
            (64607, 64598),
            (64607, 64599),
            (64607, 64600),
            (6460

In [7]:
compare = recordlinkage.Compare()

compare.string('name', 'name', label="name")
compare.exact('employees', 'employees', label='employees')
compare.string('website', 'website', method='jarowinkler', label="website")
compare.string('ticker', 'ticker', method='jarowinkler', label="ticker")
compare.string('ceo', 'ceo', method='jarowinkler', label="ceo")

features = compare.compute(pairs, companies, companies)

In [8]:
features.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 178227 entries, (0, 0) to (64607, 64607)
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   name       178227 non-null  float64
 1   employees  178227 non-null  int64  
 2   website    178227 non-null  float64
 3   ticker     178227 non-null  float64
 4   ceo        178227 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 13.2 MB


In [9]:
features.head(15)

name  employees  website  ticker  ceo
0     0       1.0          1      0.0     0.0  0.0
      20812   1.0          0      0.0     0.0  0.0
      44061   1.0          0      0.0     0.0  0.0
      62588   1.0          0      0.0     0.0  0.0
20812 0       1.0          0      0.0     0.0  0.0
      20812   1.0          0      0.0     0.0  0.0
      44061   1.0          0      0.0     0.0  0.0
      62588   1.0          0      0.0     0.0  0.0
44061 0       1.0          0      0.0     0.0  0.0
      20812   1.0          0      0.0     0.0  0.0
      44061   1.0          1      1.0     0.0  0.0
      62588   1.0          0      0.0     0.0  0.0
62588 0       1.0          0      0.0     0.0  0.0
      20812   1.0          0      0.0     0.0  0.0
      44061   1.0          0      0.0     0.0  0.0

In [10]:
matches = features[features.sum(axis=1) > 2]
print(len(matches))

18998


In [11]:
matches = features[features.sum(axis=1) > 2].reset_index()
matches['score'] = matches.loc[:, 'name':'ceo'].sum(axis=1)
matches.head(25)

,level_0,level_1,name,employees,website,ticker,ceo,score
0,44061,44061,1.0,1,1.0,0.0,0.0,3.0
1,38308,38308,1.0,1,1.0,0.0,0.0,3.0
2,6580,6580,1.0,1,0.0,0.0,1.0,3.0
3,21433,21433,1.0,1,0.0,0.0,1.0,3.0
4,21433,36906,1.0,1,0.0,0.0,1.0,3.0
5,21433,37654,1.0,1,0.0,0.0,1.0,3.0
6,36906,21433,1.0,1,0.0,0.0,1.0,3.0
7,36906,36906,1.0,1,0.0,0.0,1.0,3.0
8,36906,37654,1.0,1,0.0,0.0,1.0,3.0
9,37654,21433,1.0,1,0.0,0.0,1.0,3.0


In [12]:
companies.loc[37654,:]

Unnamed: 0                                                   102
name                                                     SIEMENS
headquarter                                              GERMANY
employees                                                10,000+
industry       INDUSTRIES            INFRASTRUCTURE          ...
website                                 HTTPS://WWW.SIEMENS.COM/
ticker                                                       NaN
ceo                                                   JOE KAESER
revenue_M                                                 102.48
marketcap_M                                              131.560
shareprice                                                   NaN
Name: 37654, dtype: object

In [13]:
companies.loc[21433,:]

Unnamed: 0                                                   111
name                                                     SIEMENS
headquarter                                              GERMANY
employees                                                10,000+
industry       INFRASTRUCTURE, TECHNOLOGY HARDWARE AND INSTRU...
website                                                      NaN
ticker                                                       NaN
ceo                                                   JOE KAESER
revenue_M                                                  6.049
marketcap_M                                                  NaN
shareprice                                                   NaN
Name: 21433, dtype: object

In [14]:
toDrop = ['name', 'employees', 'website', 'ticker', 'ceo', 'score']
matches = matches.drop(toDrop, axis=1)

In [15]:
print(matches)

       level_0  level_1
0        44061    44061
1        38308    38308
2         6580     6580
3        21433    21433
4        21433    36906
...        ...      ...
18993    64580    64580
18994    64583    64583
18995    64584    64584
18996    64593    64593
18997    64601    64601

[18998 rows x 2 columns]


In [16]:
matches = pd.MultiIndex.from_frame(matches)

In [17]:
print(matches)

MultiIndex([(44061, 44061),
            (38308, 38308),
            ( 6580,  6580),
            (21433, 21433),
            (21433, 36906),
            (21433, 37654),
            (36906, 21433),
            (36906, 36906),
            (36906, 37654),
            (37654, 21433),
            ...
            (64540, 64540),
            (64545, 64545),
            (64552, 64552),
            (64554, 64554),
            (64575, 64575),
            (64580, 64580),
            (64583, 64583),
            (64584, 64584),
            (64593, 64593),
            (64601, 64601)],
           names=['level_0', 'level_1'], length=18998)


In [18]:
print(len(matches))

18998


In [19]:
features.head()

name  employees  website  ticker  ceo
0     0       1.0          1      0.0     0.0  0.0
      20812   1.0          0      0.0     0.0  0.0
      44061   1.0          0      0.0     0.0  0.0
      62588   1.0          0      0.0     0.0  0.0
20812 0       1.0          0      0.0     0.0  0.0

In [20]:
features.head()

name  employees  website  ticker  ceo
0     0       1.0          1      0.0     0.0  0.0
      20812   1.0          0      0.0     0.0  0.0
      44061   1.0          0      0.0     0.0  0.0
      62588   1.0          0      0.0     0.0  0.0
20812 0       1.0          0      0.0     0.0  0.0

In [ ]:
classifier = recordlinkage.SVMClassifier()
classifier.fit(features, matches)

In [ ]:
print("precision: " + str(recordlinkage.precision(matches, predictions)))
print("recall: " + str(recordlinkage.recall(matches, predictions)))
print("accuracy: " + str(recordlinkage.accuracy(matches, predictions, len(test))))